In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.ensemble import RandomForestRegressor
from ipywidgets import interact
import cartopy.crs as ccrs
import netCDF4 as nc

In [ ]:
time_list = ['00','01','02','03','04','05','06','07','08','09','10','11','12',
             '13','14','15','16','17','18','19','20','21','22','23']

region_sum = 0

In [ ]:
#getting mean for the diurnal cycle
region_sum = 0

for i in range(24):
    selected_hr = 3

    day_preds = pd.read_csv('C:\\Users\\conor\\General_College\\UTRECHT ADS\\Thesis\\Data\\rescale_dfs\\june_2018\\june_2018_' + time_list[selected_hr] + 'am_ERA_preds.csv')
    day_preds = day_preds['flux_preds']
    jun18_preds = np.array(day_preds).reshape((44,74))
    jun18_preds = np.ma.array(jun18_preds, mask = NA_mask_1deg_cyc[1])
    jun18_preds = jun18_preds*10**(-6)
    
    region_sum += np.sum(jun18_preds)

In [ ]:
@interact(selected_hr = (0,23))
def f(selected_hr = 11):

    day_preds = pd.read_csv('june_2018_' + time_list[selected_hr] + 'am_ERA_preds.csv')
    day_preds = day_preds['flux_preds']
    jun18_preds = np.array(day_preds).reshape((44,74))
    jun18_preds = np.ma.array(jun18_preds, mask = NA_mask_1deg_cyc[1])
    jun18_preds = jun18_preds*10**(-6) #converting to mol.. from micro mol
    jun18_preds -= region_sum/(24*1192) #subtracting FLUXNET mean
    jun18_preds += -1.1998529233037303e-06 # adding in the biological prediction for that month

    nlats = np.shape(jun18_preds)[0]
    nlons = np.shape(jun18_preds)[1]
    lats = np.linspace((16*np.pi)/180,(60*np.pi)/180,nlats) # conversion of degree range to radians
    lons = np.linspace(-0.75*np.pi,(-61*np.pi)/180,nlons)
    lons,lats = np.meshgrid(lons,lats)
    lats = np.rad2deg(lats)
    lons = np.rad2deg(lons) 

    plt.figure(figsize=(15, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    im = ax.contourf(lons,lats,jun18_preds,transform=ccrs.PlateCarree(),cmap='nipy_spectral',
                     levels = np.linspace(-2.5e-6, 0,   16),
                     interpolation = True, corner_mask = True, extend = 'both')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=2,color='gray',alpha=0.5,linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    plt.colorbar(im,label=r'$mol m^{-2} s^{-1}$')
    ax.coastlines()
    plt.xlim((-140,-60))
    plt.ylim((15,60))
    plt.title('Biological Flux Predictions for ' + time_list[selected_hr] +  ':00 June 2018' )
    # plt.savefig('bioflux' + time_list[selected_hr] + 'map_wbar.jpg',
    #                dpi=300, bbox_inches='tight')



    plt.show()